<a href="https://colab.research.google.com/github/fourmuskees/HackathonNexarc/blob/nexarc4m/HackaThonNexarc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
#importedLibs
# Reading the data from the Excel:
data = pd.read_csv("demand_inventory.csv")

print(data.columns)
print(data.head())


Index(['SI', 'Date', 'Product_ID', 'Demand', 'Inventory'], dtype='object')
   SI        Date Product_ID  Demand  Inventory
0   0  01-02-2024         P1      51       5500
1   1  02-02-2024         P1     141       5449
2   2  03-02-2024         P1     172       5308
3   3  04-02-2024         P1      91       5136
4   4  05-02-2024         P1     198       5045


In [ ]:
fig_demand = px.line(data, x='Date',
                     y='Demand',
                     title='Demand Over Time')
fig_demand.show()

In [ ]:
fig_inventory = px.line(data, x='Date',
                        y='Inventory',
                        title='Inventory Over Time')
fig_inventory.show()

In [ ]:
# Set index as date
data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y')
data = data.set_index('Date')

In [ ]:
# Building model with optimized parameters
order = (1, 1, 1)
seasonal_order = (0, 1, 1, 7)
model = SARIMAX(data['Demand'], order=order, seasonal_order=seasonal_order)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.



In [ ]:
# Fitting model
model_fit = model.fit()

In [ ]:
# Making forecast
forecast = model_fit.forecast(steps=30)

# Printing demand_forecast
print(forecast)

62     89.792190
63    113.538447
64    149.086670
65    107.807899
66    102.361705
67    151.329615
68    125.815810
69     91.713991
70    113.615897
71    149.008752
72    107.716893
73    102.269596
74    151.237413
75    125.723601
76     91.621781
77    113.523687
78    148.916541
79    107.624683
80    102.177386
81    151.145203
82    125.631390
83     91.529571
84    113.431477
85    148.824331
86    107.532473
87    102.085176
88    151.052993
89    125.539180
90     91.437361
91    113.339267
Name: predicted_mean, dtype: float64


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [ ]:
#Inventory Optimization#

# Fitting SARIMAX model
model = SARIMAX(data['Demand'], order=(1,1,1), seasonal_order=(0,1,1,7))
model_fit = model.fit()

# Forecasting demand
forecast = model_fit.forecast(steps=30)

# Setting parameters
initial_inventory = 5500
lead_time = 7
service_level = 0.9
holding_cost = 1
stockout_cost = 5

# Inventory optimization
z = abs(np.percentile(forecast, 100 * (1 - service_level)))
order_qty = np.ceil(forecast.mean() + z).astype(int)
reorder_pt = forecast.mean() * lead_time + z
safety_stock = reorder_pt - forecast.mean() * lead_time

holding_cost = holding_cost * (initial_inventory + 0.5 * order_qty)
stockout_cost = stockout_cost * max(0, forecast.mean() * lead_time - initial_inventory)
total_cost = holding_cost + stockout_cost

print("Optimal Order Quantity:", order_qty)
print("Reorder Point:", reorder_pt)
print("Safety Stock:", safety_stock)
print("Total Cost:", total_cost)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.



Optimal Order Quantity: 211
Reorder Point: 923.7797891118508
Safety Stock: 91.61256033629377
Total Cost: 5605.5


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

